## Aquiring Data

Using ravelry.com's api.  Limit of 100k per call.  As of Friday November 19, 665,465 knitting patterns. Api does not currently allow for sort-by searches directly.  Will need to segment and pull out in sequential chunks, then filter down after the fact. 

#### PLAN OF ATTACK
* get pattern categories list
* start with one category
* then iterate through grab patterns for each if <100k
* combine into one monster dataframe

* write a script to interate through all id's grab info, write to csv... append with all info

In [1]:
import pandas as pd
import numpy as np

import ast
import json
import requests
from requests.auth import HTTPBasicAuth
from pprint import pprint

from config import basic_auth_username, basic_auth_password
from config import basic_auth_username_read_only, basic_auth_password_read_only

### Try current user call (my account)

In [459]:
def search_current_user():
    """ returns current (my) user metadata """
    response =requests.get('https://api.ravelry.com/current_user.json', auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
    if response.status_code != 200:
        print("Something's not quite right - please take a look, status_code = {}".format(response.status_code))
    else:
        return response.json()
    
search_current_user()

{'user': {'id': 264731,
  'username': 'upsidedown',
  'tiny_photo_url': 'https://avatars-d.ravelrycache.com/upsidedown/122038936/IMG_2763_tiny.JPG',
  'small_photo_url': 'https://avatars-d.ravelrycache.com/upsidedown/122038936/IMG_2763_small.JPG',
  'photo_url': 'https://avatars-d.ravelrycache.com/upsidedown/122038936/IMG_2763_large.JPG',
  'large_photo_url': 'https://avatars-d.ravelrycache.com/upsidedown/122038936/IMG_2763_xlarge.JPG'}}

In [460]:
# response =requests.get('https://api.ravelry.com/projects/DonnaYB/list.json?sort=rating_', auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
# response.json()

### More general Search

In [138]:
# def search(params, endpoint):
#         """ gets data from ravelry api with given end point (string) and parameters (dictionary)
#         outlined in the api documentation """
#         base_url = 'https://api.ravelry.com/'
#         params = params
#         response =requests.get('https://api.ravelry.com/'+endpoint, params=params,auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
#         if response.status_code != 200:
#             print("Something's not quite right - please take a look, status_code = {}".format(response.status_code))
#         return(response.json())

In [157]:
endpoint = 'patterns/search.json'
params = {'query' : {'craft':'knitting'},"page": 1,"page_size": 500}# 'year published': '2020'
# params = {'query' : {'craft':'knitting', 'sort by': 'best match'}, 'limit':500, "page": 1,"page_size": 100}
response =requests.get('https://api.ravelry.com/'+end_point, params=params, auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
# response =requests.get('https://api.ravelry.com/search.json?patterns/knitting', auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))

In [159]:
results = response.json()
results
pattern_id = []
pattern_name =[]
for i in range(len(results['patterns'])):
    pattern_id.append(results['patterns'][i]['id'])
    pattern_name.append(results['patterns'][i]['name'])
len(pattern_id)
# results['patterns'][1]
# pattern_name[0:15]
pprint(results)

500

In [ ]:
# create pandas df
# save to csv

In [60]:
import requests

result = requests.get(
#     "patterns/search"
    'https://api.ravelry.com/patterns.json',
    data={
        "query": "",
        "page": 1,
        "page_size": 100
    },
    auth=HTTPBasicAuth(basic_auth_username, basic_auth_password)
)

In [61]:
pprint(result)

<Response [400]>


## Grab actual patterns

In [2]:
# For knitting search:
response =requests.get('https://api.ravelry.com/patterns/search.json?craft=knitting&rating=4&sort=created_&page_size=100',auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
search_result = response.json()

ids = []
for i in range(len(search_result['patterns'])):
    ids.append(search_result['patterns'][i]['id'])
    
id_list = [str(x) for x in ids]
spaced_ids = '+'.join(id_list)
spaced_ids

response =requests.get(f'https://api.ravelry.com/patterns.json?ids={spaced_ids}', auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
patterns = response.json()


In [355]:
# user
response =requests.get(f'https://api.ravelry.com/people/264731.json', auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
users = response.json()
users

In [350]:
# {'paginator': {'last_page': 1000,
#                'page': 1,
#                'page_count': 1000,
#                'page_size': 100,
#                'results': 100000},d


def get_all_patterns():
     try:
        for page in range(1:1001):
             search_results = get_page(page)
             patterns = get_patterns(search_results)
             parsed_patterns = parse_patterns(patterns)
             save_patterns(parsed_patterns)
     except:
         print('Stopped on page {}'.format(page))




### For Pattern Details
#### For a singular result:

In [414]:
# For one result:
response =requests.get('https://api.ravelry.com/patterns/573.json',auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
patterns = response.json()
# pprint(patterns)
# 'Mid-calf' 'Socks','Feet / '
# Accessories
# 'Legs'

In [ ]:
def get_categories(pattern):
    cat_dict = pattern['pattern_categories'][0]
    cat_list = [cat_dict['permalink']]
    new_dict = cat_dict['parent']
    while 'parent' in new_dict.keys():
        cat_list.append(new_dict['permalink'])
        new_dict = new_dict['parent']
    if len(cat_list)>1:
        cat_list = cat_list[:2]
#     if 'yarn_weight' in pattern.keys():
#         yarn_weight = '-'.join(pattern['yarn_weight']['name'].split(' '))
#     else:
#         yarn_weight = None
#     attr_dict = {'yarn_weight':yarn_weight,
#     'pattern_attributes': [attr['permalink'] for attr in pattern['pattern_attributes']],
#     'pattern_categories':cat_list}
    return attr_dict

#### For more than one result:

In [ ]:
'pattern_categories': [{'id': 885,
                                     'name': 'Mid-calf',
                                     'parent': {'id': 354,
                                                'name': 'Socks',
                                                'parent': {'id': 353,
                                                           'name': 'Feet / '
                                                                   'Legs',
                                                           'parent': {'id': 337,
                                                                      'name': 'Accessories',
                                                                      'parent': {'id': 301,
                                                                                 'name': 'Categories',
                                                                                 'permalink': 'categories'},
                                                                      'permalink': 'accessories'},
                                                           'permalink': 'feet-legs'},
                                                'permalink': 'socks'},
                                     'permalink': 'mid-calf'}],
             'pattern_needle_sizes': [{'crochet': False,
                                       'hook': None,
                                       'id': 21,
                                       'knitting': True,
                                       'metric': 2.5,
                                       'name': 'US 1½ - 2.5 mm',
                                       'pretty_metric': '2.5',
                                       'us': '1½',
                                       'us_steel': None}],

In [229]:
pattern = response.json()
page_count = pattern['paginator']['page_count']
last_page = pattern['paginator']['last_page']
# pprint(pattern['patterns'])

In [5]:
# pprint(pattern)

In [3]:
# Note this only works for multiple call 
pattern = response.json()

id_ = []
name = []
favorites_count = []
projects_count = []
difficulty_average = []
difficulty_count = []
rating_average = []
rating_count = []
pattern_type_id = []
pattern_type_names = []
pattern_type_clothing = []
yarn_weight = []
photos_url =[]
craft_id = []
yarn_weight = []
pattern_needle_sizes = []
yardage = []
yarn_weight_description = []
yardage_max = []
yardage = []
published = []
gauge = []
gauge_pattern = []
gauge_divisor = []
row_gauge = []
free = []
downloadable = []
queued_projects_count =[]
attributes = []
generally_available = []

yarn_weight_id = []
yarn_weight_name = []
yarn_weight_ply = []
yarn_weight_knit_gauge = []
yardage_description =[]
url = []
yarn_weight_wpi = []
pattern_attributes = []
categories = []
name_permalink =[]

# parse json response 
for i in pattern['patterns']:
    id_.append(pattern['patterns'][i]['id'])
    name.append(pattern['patterns'][i]['name'])
    name_permalink.append(pattern['patterns'][i]['permalink'])
    favorites_count.append(pattern['patterns'][i]['favorites_count'])
    projects_count.append(pattern['patterns'][i]['projects_count'])
    difficulty_average.append(pattern['patterns'][i]['difficulty_average'])
    difficulty_count.append(pattern['patterns'][i]['difficulty_count'])
    queued_projects_count.append(pattern['patterns'][i]['queued_projects_count'])
    rating_average.append(pattern['patterns'][i]['rating_average'])
    rating_count.append(pattern['patterns'][i]['rating_count'])
    downloadable.append(pattern['patterns'][i]['downloadable'])
    free.append(pattern['patterns'][i]['free'])
#     row_gauge.append(pattern['patterns'][i]['row_gauge'])
    gauge_divisor.append(pattern['patterns'][i]['gauge_divisor'])
#     gauge_pattern.append(pattern['patterns'][i]['gauge_pattern'])
    gauge.append(pattern['patterns'][i]['gauge'])
#     published.append(pattern['patterns'][i]['published'])
    yardage.append(pattern['patterns'][i]['yardage'])
    yardage_max.append(pattern['patterns'][i]['yardage_max'])
    yarn_weight_description.append(pattern['patterns'][i]['yarn_weight_description'])
    generally_available.append(pattern['patterns'][i]['generally_available'])
    
    
    try:
        pattern_needle_sizes.append(pattern['patterns'][i]['pattern_needle_sizes'])
#         pattern_needle_sizes.append(ast.literal_eval(pattern['patterns'][i]['pattern_needle_sizes']))#['metric'])
    except:
        pattern_needle_sizes.append('None')
                                    
    attributes = []
    try:
        for j in range(len(pattern['patterns'][i]['pattern_attributes'])):
            attributes.append(pattern['patterns'][i]['pattern_attributes'][j]['permalink'])
        pattern_attributes.append(attributes)
    except:
        pattern_attributes.append('None')
    
    
    try:
        category_dict = pattern['patterns'][i]['pattern_categories'][0]
        category_list = [category_dict['permalink']]
        new_dict = category_dict['parent']
        while 'parent' in new_dict.keys():
            cat_list.append(new_dict['permalink'])
            new_dict = new_dict['parent']
        categories.append(cat_list)
    except:
        print("uhoh - check out categories!")
'pattern_categories': [{'id': 350,
                             'name': 'Shawl / Wrap',
                             'parent': {'id': 338,
                                        'name': 'Neck / Torso',
                                        'parent': {'id': 337,
                                                   'name': 'Accessories',
                                                   'parent': {'id': 301,
                                                              'name': 'Categories',
                                                              'permalink': 'categories'},
                                                   'permalink': 'accessories'},
                                        'permalink': 'neck-torso'},
                             'permalink': 'shawl-wrap'}],
        
    yardage_description.append(pattern['patterns'][i]['yardage_description'])
#     url.append(pattern['patterns'][i]['url'])
                      
#     craft_id.append(pattern['patterns'][i]['craft']['id'])
    photos_url.append(pattern['patterns'][i]['photos'][0]['square_url'])
    pattern_type_clothing.append(pattern['patterns'][i]['pattern_type']['clothing'])
    pattern_type_names.append(pattern['patterns'][i]['pattern_type']['permalink'])
#     pattern_type_id.append(pattern['patterns'][i]['pattern_type']['id'])
    



# pattern['patterns']['600']['photos']['image_url']

uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check out categories!
uhoh - check o

In [478]:
data = {'pattern_id':id_,
        'name':name,
        'name_permalink':name_permalink,
        'favorites_count': favorites_count,
        'projects_count': projects_count, 
        'difficulty_average' : difficulty_average, 
        'difficulty_count': difficulty_count, 
        'rating_average': rating_average,
        'queued_projects_count': queued_projects_count,
        'rating_count':rating_count,
#         'pattern_type_id':pattern_type_id,
        'pattern_type_names' :pattern_type_names,
        'pattern_type_clothing' :pattern_type_clothing,
        'photos_url' :photos_url,
#         'craft_id':craft_id,
#         'url' :url,
        'pattern_needle_sizes' :pattern_needle_sizes,
        'pattern_attributes':pattern_attributes,
        'yardage_max' :yardage_max,
        'yardage' :yardage,
#         'yardage_description':yardage_description,
        'generally_available':generally_available,
#         'published' :published,
        'gauge' :gauge,
#         'gauge_pattern' :gauge_pattern,
        'gauge_divisor' :gauge_divisor,
#         'row_gauge' :row_gauge,
        'free' :free,
        'downloadable': downloadable,
        'categories':categories,
        'yarn_weight_description' :yarn_weight_description,
#         'yarn_weight_id':yarn_weight_id,
#         'yarn_weight_name':yarn_weight_name,
#         'yarn_weight_ply':yarn_weight_ply,
#         'yarn_weight_wpi':yarn_weight_wpi,
#         'yarn_weight_knit_gauge':yarn_weight_knit_gauge,
       
       }
patterns_df = pd.DataFrame(data)

In [479]:
patterns_df.columns

Index(['pattern_id', 'name', 'name_permalink', 'favorites_count',
       'projects_count', 'difficulty_average', 'difficulty_count',
       'rating_average', 'queued_projects_count', 'rating_count',
       'pattern_type_names', 'pattern_type_clothing', 'photos_url',
       'pattern_needle_sizes', 'pattern_attributes', 'yardage_max', 'yardage',
       'generally_available', 'gauge', 'gauge_divisor', 'free', 'downloadable',
       'categories', 'yarn_weight_description'],
      dtype='object')

In [480]:
def filter_and_save_csv(df):
    # remove any non-knitting project 
#     df = df[df['craft_id'] == 2]
    # remove any project with less than 5 projects
    
    # remove any with rating less than equal 3.5
    
    # do the actual saving
    df.to_csv('data/patterns_df.csv', index=False)
    
filter_and_save_csv(patterns_df)

In [12]:
# try:
categories = []
#     category_dict = pattern['patterns'][i]['pattern_categories'][0]
category_dict= {'id': 350,
             'name': 'Shawl / Wrap',
             'parent': {'id': 338,
                        'name': 'Neck / Torso',
                        'parent': {'id': 337,
                                   'name': 'Accessories',
                                   'parent': {'id': 301,
                                              'name': 'Categories',
                                              'permalink': 'categories'},
                                   'permalink': 'accessories'},
                        'permalink': 'neck-torso'},
             'permalink': 'shawl-wrap'}
category_list = [category_dict['permalink']]
new_dict = category_dict['parent']
while 'parent' in new_dict.keys():
    category_list.append(new_dict['permalink'])
    new_dict = new_dict['parent']
    categories.append(category_list)
# except:
#     print("uhoh - check out categories!")
    
    




In [13]:
category_list

['shawl-wrap', 'neck-torso', 'accessories']